In [ ]:
#Import the necessary libraries first

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
import cv2
import shutil
import glob

In [ ]:
data = '../input/brain-mri-images-for-brain-tumor-detection/'
No_brain_tumor = '../input/brain-mri-images-for-brain-tumor-detection/no/'
Yes_brain_tumor = '../input/brain-mri-images-for-brain-tumor-detection/yes/'

In [ ]:
dirlist = [No_brain_tumor , Yes_brain_tumor]
classes = ['No','Yes']
filepaths  = []
labels = []
for i,j in zip(dirlist,classes):
    filelist = os.listdir(i)
    for f in filelist:
        filepath = os.path.join(i,f)
        filepaths.append(filepath)
        labels.append(j)
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))

In [ ]:
Files = pd.Series(filepaths,name = 'filepath')
Label = pd.Series(labels,name = 'labels')
df = pd.concat([Files,Label],axis=1)
df=pd.DataFrame(np.array(df).reshape(253,2), columns = ['filepaths', 'labels'])
df.head()

In [ ]:
print(df['labels'].value_counts())

In [ ]:
plt.figure(figsize=(12,10))
for i in range(15):
    random = np.random.randint(1,len(df))
    plt.subplot(3,5,i+1)
    plt.imshow(cv2.imread(df.loc[random,"filepaths"]))
    plt.title(df.loc[random, "labels"], size = 15, color = "black") 
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size=0.95, random_state=0)
train_new, valid = train_test_split(train, train_size=0.90, random_state=0)

print(f"train set shape: {train_new.shape}")
print(f"test set shape: {test.shape}")
print(f"validation set shape: {valid.shape}")

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,
                                   zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(dataframe=train_new,x_col="filepaths",y_col="labels",target_size=(150,150),batch_size=16,class_mode='binary',shuffle=True)


In [ ]:
val_gen= train_datagen.flow_from_dataframe(valid,x_col="filepaths",y_col="labels",target_size=(150,150),batch_size=16,class_mode='binary',shuffle=True)


In [ ]:
test_gen = test_datagen.flow_from_dataframe(test,x_col="filepaths",y_col="labels",target_size=(150,150),batch_size=16,class_mode='binary',shuffle=True)

In [ ]:
train_gen.class_indices

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet',input_shape=(200,200,3),include_top=False)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()


In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint("Tumor_classifier_model.h5",save_best_only=True,verbose=0),tf.keras.callbacks.EarlyStopping(patience=3,monitor = 'val_loss',verbose=1)]

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=Adam(lr = 0.0001),metrics=['accuracy'])

history = model.fit(train_gen,validation_data=val_gen,epochs = 40,callbacks=[callbacks],verbose=1)

In [ ]:
model.load_weights("Tumor_classifier_model.h5")
model.evaluate(test_gen)